In [20]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt 

In [21]:
PATH = "./data/"

In [22]:
df_0 = pd.read_csv(PATH + "train-metadata.csv")

/tmp/ipykernel_1850/1877868789.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv(PATH + "train-metadata.csv")


In [23]:
# Clean and preprocess data for training and testing 

cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
df_0.drop(columns=cols_to_drop, inplace=True)
df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})

# replace all NaNs with np.nan
df_0.replace("", np.nan, inplace=True)

# Drop any column with a string vallue 
for col in df_0.columns:
    if df_0[col].dtype == "object":
        df_0.drop(columns=[col], inplace=True)

# clean up for training, split into train and test
train_frac = 0.8 
train_size = int(train_frac * len(df_0))

train = df_0[:train_size]
x_train = df_0.drop(columns=["target"])
y_train = df_0["target"]

test = df_0[train_size:]
x_test = df_0.drop(columns=["target"])
y_test = df_0["target"]

In [24]:
# Define the NN 

inputs = keras.Input(shape=(len(x_train.keys()),)) # One input for each feature in the dataset 

# Add two hidden layers with 64 units each and ReLU activation
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

# Define the output layer with a single unit (suitable for regression tasks)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="isic-v0")

In [25]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10
5014/5014 [==============================] - 7s 1ms/step - loss: 0.0261 - binary_accuracy: 0.9989 - val_loss: 0.0067 - val_binary_accuracy: 0.9992
Epoch 2/10
5014/5014 [==============================] - 6s 1ms/step - loss: 0.0084 - binary_accuracy: 0.9990 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
Epoch 3/10
5014/5014 [==============================] - 6s 1ms/step - loss: 0.0084 - binary_accuracy: 0.9990 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
Epoch 4/10
5014/5014 [==============================] - 6s 1ms/step - loss: 0.0083 - binary_accuracy: 0.9990 - val_loss: 0.0067 - val_binary_accuracy: 0.9992
Epoch 5/10
5014/5014 [==============================] - 6s 1ms/step - loss: 0.0083 - binary_accuracy: 0.9990 - val_loss: 0.0068 - val_binary_accuracy: 0.9992
Epoch 6/10
5014/5014 [==============================] - 6s 1ms/step - loss: 0.0083 - binary_accuracy: 0.9990 - val_loss: 0.0066 - val_binary_accuracy: 0.9992
Epoch 7/10
5014/5014 [==============================

In [29]:
test_df = pd.read_csv(PATH + "test-metadata.csv")
isic_ids = test_df["isic_id"]

try: 
    test_df.drop(columns=cols_to_drop, inplace=True)
except KeyError: 
    pass

test_df.replace("", np.nan, inplace=True)
test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})

# Drop any column with a string vallue 
for col in test_df.columns:
    if test_df[col].dtype == "object":
        test_df.drop(columns=[col], inplace=True)

In [34]:
preds[:,0]

array([-7.7168064, -7.7168064, -7.7168064], dtype=float32)

In [35]:
preds = model.predict(test_df)

df_sub = pd.DataFrame({"isic_id": isic_ids, "target": preds[:,0]})
df_sub.to_csv("submission.csv", index=False)


1/1 [==============================] - 0s 16ms/step
